<a href="https://colab.research.google.com/github/aflores/colab-notebooks/blob/master/gsheetrefug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Sheet refugee data generator

This notebook was used to generate test data for the reportermate exercise
Here are some of the this we did here:

- Read data from a Google Sheets document
- Used the above data to generate additional information (IRA -> PAK)
  - Find/repalce the vaule of some 'string' columns
  - Change the value of a couple numeric columns (lambda functions)
- Concatenate the resulting DataFrames
- Take a peek at the resulting data using a client-side pivot table
- export the combined data as JSON and CSV

In [114]:
import gspread # https://gspread.readthedocs.io/en/latest/index.html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()

#import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [115]:
data_url = 'https://docs.google.com/spreadsheets/d/1LMB3gTs59_sbsHxzSMBujKDuuN55sWhSW2fERUO9a-8/edit?usp=sharing'

In [116]:
spreadsheet = gc.open_by_url(data_url)
worksheet = spreadsheet.get_worksheet(0)

In [117]:
df = pd.DataFrame(worksheet.get_all_records())

In [118]:
df.head()

,date,country,border,type,returns,assisted
0,2019-01-01,IRA,Herat,spn,8000,300
1,2019-01-01,IRA,Herat,dept,6300,400
2,2019-01-01,IRA,Nimtoz,spn,3000,35
3,2019-01-01,IRA,Nimtoz,dept,6000,658
4,2019-02-01,IRA,Herat,spn,8000,300


In [119]:
# clone the data to create the PAK information
df_pak = df.copy()

# replace country and borders
df_pak['country'] = 'PAK'
df_pak.loc[(df_pak.border == 'Herat'),'border'] = 'Turkham' 
df_pak.loc[(df_pak.border == 'Nimtoz'),'border'] = 'Kandahar' 

# change the numbers
df_pak['returns'] = df_pak['returns'].apply(lambda returns: np.floor(returns/7)) 
df_pak['assisted'] = df_pak['assisted'].apply(lambda assisted: np.floor(assisted/5))


In [120]:
df_combo = pd.concat([df,df_pak],ignore_index=True)
#df_combo.head(300)
df_combo.groupby(['country','type','border']).sum()

returns  assisted
country type border                     
IRA     dept Herat     58300.0    3600.0
             Nimtoz    54000.0    5922.0
        spn  Herat     71500.0    2700.0
             Nimtoz    30000.0     350.0
PAK     dept Kandahar   7713.0    1179.0
             Turkham    8324.0     720.0
        spn  Kandahar   4280.0      70.0
             Turkham   10207.0     540.0

In [121]:
!pip install pivottablejs

from pivottablejs import pivot_ui
from IPython.display import HTML
from IPython.display import IFrame


In [122]:
pivot_ui(df_combo, outfile_path='pivottablejs.html')
HTML('pivottablejs.html')

In [127]:
#df_combo.to_json(orient='records')
df_combo.to_csv(index=False)

'date,country,border,type,returns,assisted\n2019-01-01,IRA,Herat,spn,8000.0,300.0\n2019-01-01,IRA,Herat,dept,6300.0,400.0\n2019-01-01,IRA,Nimtoz,spn,3000.0,35.0\n2019-01-01,IRA,Nimtoz,dept,6000.0,658.0\n2019-02-01,IRA,Herat,spn,8000.0,300.0\n2019-02-01,IRA,Herat,dept,6500.0,400.0\n2019-02-01,IRA,Nimtoz,spn,3000.0,35.0\n2019-02-01,IRA,Nimtoz,dept,6000.0,658.0\n2019-02-01,IRA,Herat,spn,8000.0,300.0\n2019-02-01,IRA,Herat,dept,6500.0,400.0\n2019-01-10,IRA,Nimtoz,spn,3000.0,35.0\n2019-01-10,IRA,Nimtoz,dept,6000.0,658.0\n2019-01-10,IRA,Herat,spn,8000.0,300.0\n2019-01-10,IRA,Herat,dept,6500.0,400.0\n2019-01-10,IRA,Nimtoz,spn,3000.0,35.0\n2019-01-10,IRA,Nimtoz,dept,6000.0,658.0\n2019-02-10,IRA,Herat,spn,8000.0,300.0\n2019-02-10,IRA,Herat,dept,6500.0,400.0\n2019-02-10,IRA,Nimtoz,spn,3000.0,35.0\n2019-02-10,IRA,Nimtoz,dept,6000.0,658.0\n2019-02-10,IRA,Herat,spn,7500.0,300.0\n2019-02-10,IRA,Herat,dept,6500.0,400.0\n2019-02-10,IRA,Nimtoz,spn,3000.0,35.0\n2019-02-10,IRA,Nimtoz,dept,6000.0,658.0\n20